In [2]:
# Cell 0 — Method 2 (SegFormer) 配置 + 路径（用你给的5个路径）
from pathlib import Path
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

X_TEST_DIR  = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\X_test_xNbnvIa")
X_TRAIN_DIR = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\X_train_uDRk9z9")
X_UNLAB_DIR = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\X_unlabeled_mtkxUlo")
Y_TRAIN_CSV = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\Y_train_T9NrBYo.csv")
SAMPLE_SUB  = Path(r"C:\Users\asus\Desktop\ECN\DEEP\DataChallenge\data\submission_csv_file_random_example_3qPSCtv.csv")

OUT_DIR = Path(r"exp_outputs\Exp02_SegFormer_keep272_pad288")
OUT_DIR.mkdir(parents=True, exist_ok=True)

NUM_CLASSES = 3
IGNORE_INDEX = 255
H = 160
W_PAD = 288  # 为 SegFormer stride 更友好（32的倍数），推理/导出再裁回 160/272



DEVICE: cuda


In [3]:
# Cell 1 — 工具函数：解析名字 / 读取X / 读取Y（mask）/ padding
NAME_RE = re.compile(r"well_(\d+)_section_(\d+)_patch_(\d+)$")

def parse_name(stem: str):
    m = NAME_RE.match(stem)
    if not m:
        raise ValueError(stem)
    return int(m.group(1)), int(m.group(2)), int(m.group(3))

def list_npy_once(dir_path: Path):
    # Windows 下 *.npy 已经能匹配 .NPY，避免重复计数
    return sorted(list(dir_path.rglob("*.npy")))

def load_x(path: Path) -> np.ndarray:
    x = np.load(path)
    if x.ndim == 3 and x.shape[0] == 1:
        x = x[0]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx > mn:
        x = (x - mn) / (mx - mn)
    else:
        x = np.zeros_like(x, dtype=np.float32)
    return x  # (160,w)

def pad_x_to_288(x: np.ndarray) -> np.ndarray:
    h, w = x.shape
    out = np.zeros((h, W_PAD), dtype=np.float32)
    out[:, :w] = x
    return out

y_df = pd.read_csv(Y_TRAIN_CSV, index_col=0)

def restore_mask_from_row(row_values: np.ndarray) -> np.ndarray:
    vals = row_values[row_values != -1]
    return vals.reshape(160, -1).astype(np.int64)  # (160,160) or (160,272)

def pad_mask_to_288(mask: np.ndarray, w: int) -> np.ndarray:
    out = np.full((H, W_PAD), IGNORE_INDEX, dtype=np.int64)
    out[:, :w] = mask
    return out

def make_valid_mask(w: int) -> np.ndarray:
    valid = np.zeros((H, W_PAD), dtype=np.bool_)
    valid[:, :w] = True
    return valid


In [4]:
# Cell 2 — manifest（train / unlabeled / test）
def build_manifest(x_dir: Path) -> pd.DataFrame:
    rows = []
    for p in list_npy_once(x_dir):
        stem = p.stem
        well, section, patch = parse_name(stem)
        arr = np.load(p, mmap_mode="r")
        if arr.ndim == 3 and arr.shape[0] == 1:
            w = int(arr.shape[2])
        elif arr.ndim == 2:
            w = int(arr.shape[1])
        else:
            raise ValueError(arr.shape)
        rows.append({"name": stem, "well": well, "section": section, "patch": patch, "w": w, "path": str(p)})
    return pd.DataFrame(rows)

train_df = build_manifest(X_TRAIN_DIR)
unlab_df = build_manifest(X_UNLAB_DIR)
test_df  = build_manifest(X_TEST_DIR)

# 只保留有标签的训练patch
train_df = train_df[train_df["name"].isin(y_df.index)].reset_index(drop=True)

print("train:", len(train_df), "w:", train_df["w"].value_counts().to_dict(), "wells:", sorted(train_df["well"].unique()))
print("unlab:", len(unlab_df), "w:", unlab_df["w"].value_counts().to_dict())
print("test :", len(test_df),  "w:", test_df["w"].value_counts().to_dict())


train: 4410 w: {272: 3726, 160: 684} wells: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]
unlab: 1980 w: {160: 1476, 272: 504}
test : 972 w: {272: 972}


In [13]:
# Cell 3 — Dataset + DataLoader（按 well 划分 val）
from torch.utils.data import Dataset, DataLoader

class SegDataset288(Dataset):
    def __init__(self, df: pd.DataFrame, y_df: pd.DataFrame, train_mode: bool):
        self.df = df.reset_index(drop=True)
        self.y_df = y_df
        self.train_mode = train_mode

    def __len__(self): 
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        name = row["name"]
        w = int(row["w"])

        x = load_x(Path(row["path"]))          # (160,w)
        x = pad_x_to_288(x)                    # (160,288)

        # 简单增强（可删）
        if self.train_mode:
            if np.random.rand() < 0.5:
                x = np.flip(x, axis=1).copy()

        x_t = torch.from_numpy(x).unsqueeze(0)  # (1,160,288)
        x_t = x_t.repeat(3, 1, 1)               # (3,160,288)


        m = restore_mask_from_row(self.y_df.loc[name].values)  # (160,w)
        y = pad_mask_to_288(m, w=w)                            # (160,288) padding 区 IGNORE
        if self.train_mode:
            if x.shape[1] == W_PAD and np.random.rand() < 0.5:
                y = np.flip(y, axis=1).copy()

        y_t = torch.from_numpy(y).long()
        valid_t = torch.from_numpy(make_valid_mask(w))
        meta = {"name": name, "well": int(row["well"]), "orig_w": w}
        return x_t, y_t, valid_t, meta

VAL_WELLS = {5}  # 你也可以换成 {1} 或 {2} 等做对比
train_split = train_df[~train_df["well"].isin(VAL_WELLS)].reset_index(drop=True)
val_split   = train_df[train_df["well"].isin(VAL_WELLS)].reset_index(drop=True)

train_loader = DataLoader(SegDataset288(train_split, y_df, train_mode=True),  batch_size=8, shuffle=True,  num_workers=0)
val_loader   = DataLoader(SegDataset288(val_split,   y_df, train_mode=False), batch_size=8, shuffle=False, num_workers=0)

print("train_split:", len(train_split), "val_split:", len(val_split))


train_split: 4122 val_split: 288


In [14]:
# Cell 4 — Loss & Metric（ignore padding）
# 用你统计的像素比例做一组合理权重（也可后面调）
ce_weights = torch.tensor([1.0, 3.0, 4.0], dtype=torch.float32).to(DEVICE)
ce = nn.CrossEntropyLoss(weight=ce_weights, ignore_index=IGNORE_INDEX)

def soft_dice_loss(logits, target, smooth=1.0):
    probs = torch.softmax(logits, dim=1)
    valid = (target != IGNORE_INDEX).unsqueeze(1)
    t = target.clone()
    t[t == IGNORE_INDEX] = 0
    onehot = F.one_hot(t, num_classes=NUM_CLASSES).permute(0,3,1,2).float()
    probs = probs * valid
    onehot = onehot * valid
    inter = (probs * onehot).sum((0,2,3))
    denom = (probs + onehot).sum((0,2,3))
    dice = (2*inter + smooth) / (denom + smooth)
    return 1.0 - dice.mean()

def combo_loss(logits, target, dice_w=0.5):
    return (1-dice_w)*ce(logits, target) + dice_w*soft_dice_loss(logits, target)

def mean_iou(pred, target):
    valid = (target != IGNORE_INDEX)
    ious = []
    for c in range(NUM_CLASSES):
        p = (pred == c) & valid
        t = (target == c) & valid
        inter = (p & t).sum().float()
        union = (p | t).sum().float()
        ious.append(torch.tensor(1.0, device=pred.device) if union.item()==0 else inter/union)
    return torch.stack(ious).mean()


In [15]:
# Cell 5 — SegFormer 模型（需要 transformers）
# 如缺包：pip install transformers accelerate
from transformers import SegformerForSemanticSegmentation

BACKBONE = "nvidia/segformer-b2-finetuned-ade-512-512"  # 方法2：SegFormer（避开 ResNet34-UNet）
model = SegformerForSemanticSegmentation.from_pretrained(
    BACKBONE,
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True
).to(DEVICE)

print("model loaded:", BACKBONE)


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([3, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loaded: nvidia/segformer-b2-finetuned-ade-512-512


In [16]:
# Cell 6 — 训练（保存 best）
from tqdm import tqdm

best_path = OUT_DIR / "best.pt"
opt = torch.optim.AdamW(model.parameters(), lr=6e-5, weight_decay=0.01)

best_miou = -1.0
EPOCHS = 10  # 先跑通；后面冲分再加

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

for ep in range(1, EPOCHS+1):
    model.train()
    tr_loss, n = 0.0, 0
    for x, y, valid, meta in tqdm(train_loader, desc=f"[Exp02] train ep{ep}", leave=False):
        x = x.to(DEVICE)   # (B,1,160,288)
        y = y.to(DEVICE)   # (B,160,288)

        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            logits = model(pixel_values=x).logits  # (B,C,160,288)
            loss = combo_loss(logits, y, dice_w=0.5)

        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        tr_loss += float(loss.item()) * x.size(0)
        n += x.size(0)
    tr_loss /= max(1,n)

    model.eval()
    miou_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y, valid, meta in tqdm(val_loader, desc=f"[Exp02] val ep{ep}", leave=False):
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            logits = model(pixel_values=x).logits
            pred = torch.argmax(logits, dim=1)
            miou_sum += float(mean_iou(pred, y).item()) * x.size(0)
            n += x.size(0)
    val_miou = miou_sum / max(1,n)

    print(f"[Exp02] ep{ep:02d}/{EPOCHS}  train_loss={tr_loss:.4f}  val_mIoU={val_miou:.4f}")

    if val_miou > best_miou:
        best_miou = val_miou
        torch.save(model.state_dict(), best_path)

print("[Exp02] best mIoU:", best_miou, "saved:", best_path)


C:\Users\asus\AppData\Local\Temp\ipykernel_6176\1866584696.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
[Exp02] train ep1:   0%|          | 0/516 [00:00<?, ?it/s]C:\Users\asus\AppData\Local\Temp\ipykernel_6176\1866584696.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):


RuntimeError: input and target batch or spatial sizes don't match: target [8, 160, 288], input [8, 3, 40, 72]

In [ ]:
# Cell 7 — Exp02：重新对 test 预测（严格按样例 972 个名字），保存 npy + 生成正确 submission
import numpy as np
import pandas as pd

pred_dir = OUT_DIR / "test_predictions"
pred_dir.mkdir(parents=True, exist_ok=True)

# 读取样例顺序
sample = pd.read_csv(SAMPLE_SUB)
name_col = sample.columns[0]
ordered_names_raw = sample[name_col].astype(str).tolist()

def norm_name(s: str) -> str:
    s = str(s).strip()
    if s.lower().endswith(".npy"):
        s = s[:-4]
    return s

ordered_names = [norm_name(n) for n in ordered_names_raw]
print("sample rows:", len(ordered_names), "name_col:", name_col)

# 建 test 索引（只 glob 一次，避免重复）
test_files = sorted(list(X_TEST_DIR.rglob("*.npy")))
test_index = {p.stem: p for p in test_files}
test_index.update({p.stem.lower(): p for p in test_files})

# 加载 best
model.load_state_dict(torch.load(best_path, map_location=DEVICE))
model.eval()
print("loaded best:", best_path)

# 清空旧预测（避免混入残留）
for p in pred_dir.glob("*.npy"):
    p.unlink()

def pad_x_to_288(x: np.ndarray) -> np.ndarray:
    h, w = x.shape
    out = np.zeros((h, W_PAD), dtype=np.float32)
    out[:, :w] = x
    return out

# 推理（按样例 972 个）
with torch.no_grad():
    for name in ordered_names:
        key = name if name in test_index else name.lower()
        if key not in test_index:
            hits = list(X_TEST_DIR.rglob(f"{name}.npy"))
            if len(hits) == 0:
                raise FileNotFoundError(f"X_test 中找不到：{name}.npy")
            x_path = hits[0]
        else:
            x_path = test_index[key]

        x = load_x(x_path)
        w = x.shape[1]
        x_pad = pad_x_to_288(x)
        x_t = torch.from_numpy(x_pad).unsqueeze(0).to(DEVICE)   # (1,160,288)
        x_t = x_t.repeat(3,1,1)                                 # (3,160,288)
        x_t = x_t.unsqueeze(0)                                  # (1,3,160,288)


        logits = model(pixel_values=x_t).logits  # (1,C,160,288)
        pred = torch.argmax(logits, dim=1).squeeze(0).cpu().numpy().astype(np.int64)  # (160,288)

        # 裁回原宽（160 或 272）
        pred = pred[:, :w]
        np.save(pred_dir / f"{name}.npy", pred)

print("saved npy predictions to:", pred_dir)

# 生成 submission（完全匹配样例结构）
size_labels = 272
flat_len = 160 * size_labels
pred_map = {}

for p in pred_dir.glob("*.npy"):
    name = p.stem
    prediction = np.load(p)
    if prediction.shape[1] != size_labels:
        aux = -1 + np.zeros(flat_len, dtype=np.int64)
        aux[0:160*160] = prediction.flatten()
    else:
        aux = prediction.flatten().astype(np.int64)
    pred_map[name] = aux

missing = [n for n in ordered_names if n not in pred_map]
assert len(missing) == 0, f"仍缺预测：{missing[:5]}"

data = np.stack([pred_map[n] for n in ordered_names], axis=0)  # (972, 43520)
col_names = [str(i) for i in range(flat_len)]
sub_df = pd.DataFrame(data, columns=col_names)
sub_df.insert(0, name_col, ordered_names_raw)

out_csv = OUT_DIR / "y_test_submission_MATCH_SAMPLE.csv"
sub_df.to_csv(out_csv, index=False)
print("Saved submission:", out_csv, "shape:", sub_df.shape)
